In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-09"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9101,2024-10-09,Eua Nba,20:30,New York Knicks,Washington Wizards,1.27,3.70,222.5,1.89,1.89,-10.5,2.04,1.69,bLJpPrqD,0.787402,0.270270,0.529101,0.529101,0.057672,1.0,0.2,NaN,NaN,255.340,69.459396,0.272027,175.862,52.309562,0.297447,199.364,391.450,249.10,147.46,0.0,0.0,0.0,0.0,0.691457,0.000000,0.132701,0.65,0.130,2.076923,0.590540,0.5,-0.090540,2.04,0.408,6.617647,0.322862,0.4,0.077138
9102,2024-10-09,Eua Nba,21:00,Oklahoma City Thunder,Houston Rockets,1.51,2.53,223.5,1.82,1.88,-5.5,1.97,1.74,Ctk0RYPt,0.662252,0.395257,0.549451,0.531915,0.057509,0.4,0.8,NaN,NaN,201.584,55.728039,0.276451,158.396,19.652728,0.124073,188.898,165.042,207.90,170.63,0.0,0.0,0.0,0.0,0.357054,0.022933,0.087674,-0.03,-0.006,-85.000000,0.682191,0.8,0.117809,-0.44,-0.088,-17.386364,0.624945,0.6,-0.024945
9103,2024-10-09,Eua Nba,21:00,San Antonio Spurs,Orlando Magic,2.06,1.74,215.5,1.89,1.89,1.5,1.92,1.86,GdlVWf2J,0.485437,0.574713,0.529101,0.529101,0.060150,0.8,1.0,NaN,NaN,178.044,37.970003,0.213262,206.656,17.244335,0.083445,190.894,220.124,238.61,205.92,0.0,0.0,0.0,0.0,0.119092,0.000000,0.022448,0.60,0.120,8.833333,0.456553,0.3,-0.156553,-5.00,-1.000,-0.740000,0.513086,0.1,-0.413086
9104,2024-10-09,Eua Nba,23:30,Sacramento Kings,Golden State Warriors,1.71,2.12,220.5,1.86,1.92,-3.5,2.01,1.71,Yew9PCfg,0.584795,0.471698,0.537634,0.520833,0.056493,0.2,0.4,NaN,NaN,163.478,24.036229,0.147030,233.570,64.706234,0.277031,205.256,233.570,168.78,171.84,0.0,0.0,0.0,0.0,0.151391,0.022448,0.114049,-0.75,-0.150,-4.733333,0.528487,0.4,-0.128487,1.51,0.302,3.708609,0.476474,0.7,0.223526
9105,2024-10-09,Europa Euroliga,14:00,Monaco,Maccabi Tel Aviv,1.31,3.63,162.5,1.87,1.95,-8.5,2.00,1.71,I9G9AtoC,0.763359,0.275482,0.534759,0.512821,0.038841,0.6,0.4,NaN,NaN,144.534,40.133807,0.277677,190.564,74.014626,0.388398,127.066,217.918,145.08,132.84,0.0,0.0,0.0,0.0,0.664165,0.029617,0.110545,0.51,0.102,3.039216,0.669148,0.9,0.230852,3.89,0.778,3.380463,0.466279,0.6,0.133721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9196,2024-10-09,Europa Liga Aba,15:00,Cibona,Krka,1.49,2.50,155.5,1.82,1.84,-5.5,1.97,1.71,6NGnRSDb,0.671141,0.400000,0.549451,0.543478,0.071141,0.4,0.4,NaN,NaN,168.678,36.088674,0.213950,176.510,39.915155,0.226135,160.706,356.650,201.00,237.14,0.0,0.0,0.0,0.0,0.357984,0.007728,0.099917,1.12,0.224,2.187500,0.000000,0.0,0.000000,-5.00,-1.000,-1.500000,0.000000,0.0,0.000000
9197,2024-10-09,Europa Liga Letônia,12:30,TAL TECH,Keila Coolbet,1.07,7.22,170.5,1.82,1.88,-17.5,1.98,1.70,jkDVcUe1,0.934579,0.138504,0.549451,0.531915,0.073084,0.6,0.0,NaN,NaN,471.528,454.018052,0.962866,NaN,NaN,NaN,204.380,0.000,128.10,0.00,0.0,0.0,0.0,0.0,1.049145,0.022933,0.107603,-2.44,-0.488,-0.143443,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9198,2024-10-09,Finlândia Korisliiga Feminina,12:30,Pyrinto F,Kouvottaret F,2.41,1.47,159.5,1.84,1.82,1.5,1.88,1.78,hYJLtLL6,0.414938,0.680272,0.543478,0.549451,0.095210,0.8,0.0,NaN,NaN,155.160,41.113908,0.264977,315.508,359.803846,1.140395,182.208,0.000,175.12,152.76,0.0,0.0,0.0,0.0,0.342619,0.007728,0.038640,1.42,0.284,4.964789,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9199,2024-10-09,Kosovo Superliga,14:00,Peja,Prizreni,1.16,4.31,168.5,1.83,1.83,-10.5,1.83,1.83,vglToqCH,0.862069,0.2

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
